In [1]:
import os
%pip install pymysql
import pymysql
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def connect_db(user:str, user_pwd:str, db:str, host:str) -> pd.DataFrame:
    conn = pymysql.connect(
        host=host,
        port=3306,
        user=user,
        passwd=user_pwd,
        db=db)

    df = pd.read_sql_query("SELECT * FROM final_project.books",
    conn)
    df_copy = df.copy()
    df_copy = df_copy.drop(['id'], axis=1)
    conn.close()
    return df_copy

df_copy = connect_db('mysql_user','SxEmnfzY94$55eBQ', 'final_project', 'localhost')


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip show sklearn

Note: you may need to restart the kernel to use updated packages.


In [3]:
def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    genre_df_dummies = pd.get_dummies(df['genres'])
    df_copy_cleaned = df.drop(['author', 'description', 'language', 'coverImg', 'likedPercent', 'isbn', 'title', 'genres'], axis=1)
    df_copy_merged = pd.concat([genre_df_dummies, df_copy_cleaned], axis=1)
    return df_copy_merged

In [4]:
X = preprocess_data(df_copy)
X = X.drop(['category'], axis=1)
X

,comedy,environment,fantasy,fiction,food,health,horror,literature,nonfiction,romance,tech,thriller,war,rating,numRatings
0,0,0,0,1,0,0,0,0,0,0,0,0,0,4.33,6376780.0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,4.50,2507620.0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,4.28,4501080.0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,4.26,2998240.0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,3.60,4964520.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7238,1,0,0,0,0,0,0,0,0,0,0,0,0,4.23,44034.0
7239,0,0,0,0,0,0,0,0,0,0,0,1,0,3.76,25143.0
7240,0,0,0,1,0,0,0,0,0,0,0,0,0,3.50,27511.0
7241,0,0,0,0,0,0,0,0,0,1,0,0,0,4.04,24431.0


In [5]:
def normalize_data(data:pd.DataFrame) -> pd.DataFrame:
    min_max_scaler = MinMaxScaler()
    features_encoded = min_max_scaler.fit_transform(data)
    return features_encoded

def generate_df_for_training(encoded_data: np.ndarray):
    df_kmeans = pd.DataFrame(encoded_data, columns = ['comedy', 'environment', 'fantasy', 'fiction', 'health', 'horror',
           'literature', 'nonfiction', 'romance', 'tech', 'food', 'thriller', 'war',
           'rating', 'numRatings'])
    return df_kmeans

encoded_data = normalize_data(X)
encoded_data
df_kmeans = generate_df_for_training(encoded_data)


In [6]:
def kmeans_model_elaboration(data:pd.DataFrame):
    kmeans = KMeans(n_clusters=5, random_state=0).fit(data)
    return kmeans

In [7]:
kmeans = kmeans_model_elaboration(encoded_data)

In [8]:
def predict(data:pd.DataFrame, df_kmeans: pd.DataFrame ,encoded_data: pd.DataFrame, kmeans) -> pd.DataFrame:
    data['category'] = kmeans.predict(encoded_data)
    df_kmeans['category'] = kmeans.predict(encoded_data)
    return data

In [9]:
category_predictions = predict(df_copy, df_kmeans, encoded_data, kmeans)

In [49]:
import pickle
def nearest_neighbors_modelisation(data:pd.DataFrame):
    model = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
    model.fit(data)
    dist, idlist = model.kneighbors(data)
    with open ('./distance.sav', 'wb') as f:
        pickle.dump(dist, f)
    with open ('./idlist.sav', 'wb') as f:
        pickle.dump(idlist, f)
    
    
    df_dist = pd.DataFrame({'dist':[dist], 'idlist':[idlist]})
    test = df_dist.to_csv('distances_knn.csv', index=False)
    return df_dist

In [50]:
df_dist = nearest_neighbors_modelisation(df_kmeans)

In [52]:
with open('./distance.sav', 'rb') as f:
        dist = pickle.load(f)
with open('./idlist.sav', 'rb') as f:
        idlist = pickle.load(f)
loaded_model, idlist

(array([[0.00000000e+00, 2.68313566e-01, 4.07578270e-01, ...,
         5.50028320e-01, 5.58751319e-01, 5.62503208e-01],
        [0.00000000e+00, 3.55574424e-02, 4.01425449e-02, ...,
         8.07429407e-02, 1.39228988e-01, 1.52162572e-01],
        [0.00000000e+00, 1.40698422e-01, 1.99942558e-01, ...,
         2.82761994e-01, 2.95593331e-01, 3.10391271e-01],
        ...,
        [0.00000000e+00, 5.81919132e-05, 1.78986374e-04, ...,
         5.56004451e-03, 5.56728799e-03, 5.58000047e-03],
        [0.00000000e+00, 3.96957061e-05, 9.00623009e-05, ...,
         4.64966192e-04, 4.68523155e-04, 1.17280181e-03],
        [0.00000000e+00, 2.37178287e-04, 4.79051765e-04, ...,
         1.20766005e-03, 1.37455275e-03, 3.12130609e-03]]),
 array([[   0,    2,   10, ...,  217,    6,   59],
        [   1,  103,  105, ...,   77,  147,   42],
        [   2,   10,   80, ...,  217,   59,    6],
        ...,
        [7240, 6524, 2387, ..., 5196, 6638, 4801],
        [7241, 6272, 5192, ..., 3910, 4504, 3820

In [29]:
df_dist['dist'][0]

array([[0.00000000e+00, 2.68313566e-01, 4.07578270e-01, ...,
        5.50028320e-01, 5.58751319e-01, 5.62503208e-01],
       [0.00000000e+00, 3.55574424e-02, 4.01425449e-02, ...,
        8.07429407e-02, 1.39228988e-01, 1.52162572e-01],
       [0.00000000e+00, 1.40698422e-01, 1.99942558e-01, ...,
        2.82761994e-01, 2.95593331e-01, 3.10391271e-01],
       ...,
       [0.00000000e+00, 5.81919132e-05, 1.78986374e-04, ...,
        5.56004451e-03, 5.56728799e-03, 5.58000047e-03],
       [0.00000000e+00, 3.96957061e-05, 9.00623009e-05, ...,
        4.64966192e-04, 4.68523155e-04, 1.17280181e-03],
       [0.00000000e+00, 2.37178287e-04, 4.79051765e-04, ...,
        1.20766005e-03, 1.37455275e-03, 3.12130609e-03]])

In [ ]:
def book_recommendation_engine(book_name):
    book_list_name = []
    book_list_id = []
    book_id = df_copy[df_copy['title'].str.contains(book_name, case=False)].index
    book_id = book_id[0]
    for newid in idlist[book_id]:
        book_list_name.append(df_copy.loc[newid].title)
#     return book_list_name
    return df_copy.iloc[idlist[book_id]]

In [ ]:
book_list_name = book_recommendation_engine('American Sniper: The Autobiography of the Most Lethal Sniper in U.S. Military History')
book_list_name

In [ ]:
df_copy = df_copy.reset_index().rename(columns={'index': 'id'})
df_copy.id = df_copy.id.apply(lambda x : x+1)
df_copy

In [ ]:
df_copy

In [ ]:
from sqlalchemy import create_engine
def insert_to_db(user:str, user_pwd:str, db:str, host:str, df:pd.DataFrame):
    sqlEngine       = create_engine(f"mysql+pymysql://{user}:{user_pwd}@{host}/{db}?charset=utf8mb4")
    dbConnection    = sqlEngine.connect()
    df.to_sql(name='books', con = dbConnection, if_exists = 'replace', chunksize = 1000)
    

In [ ]:
# insert_to_db('mysql_user','SxEmnfzY94$55eBQ', 'final_project', 'localhost', df_copy)
